In [1]:
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BMOTIF

We need a namespace (`BLDG`) to name the entities that will exist in our model. `NUM_VAVs` will be used to generate a number of VAVs for this example

In [2]:
NUM_VAVs = 3
BLDG = Namespace("urn:my-building/")

Create an in-memory BuildingMOTIF instance and create a `my-building` model that we will populate by evaluating templates

In [3]:
bm = BuildingMOTIF("sqlite://")
bldg = Model.create("https://example.com")

Load in templates / classes from the Brick ontology (this may take 1-2 minutes):

In [4]:
Library.load(ontology_graph="../libraries/brick/Brick.ttl")

Library(_id=1, _name=rdflib.term.URIRef('https://brickschema.org/schema/1.3/Brick'), _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10fdd8bb0>)

We load the G36 template library and pull out the "vav-cooling-only" template, corresponding to the pointlist in Section 4.1 of Guideline 36

In [5]:
lib = Library.load(directory="../libraries/ashrae/guideline36/")
system_specifications = lib.get_shape_collection().get_shapes_of_definition_type(BMOTIF.System_Specification)
print(f"lib contains {len(system_specifications)} system specifications: {system_specifications}")
vav_templ = lib.get_template_by_name("vav-terminal-unit-cooling")

lib contains 2 system specifications: [rdflib.term.URIRef('urn:ashrae/g36/4.1/vav-cooling-only/vav-cooling-only'), rdflib.term.URIRef('urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat')]


Now that we have the template, we do the following to create the VAV:

In [6]:
# create a "name" for the VAV
vav_name = BLDG["vav-0"]
# evaluate the template with that name to make sure the VAV has that name
tmp = vav_templ.evaluate({"name": vav_name})
# check what parameters are left
print(tmp.parameters)

{'dat', 'ztemp', 'co2', 'dmp', 'occ', 'zone'}


If we don't want to provide names for these yet, we can call `.fill()` to invent names for them. This is helpful for testing and prototyping

In [7]:
bindings, vav_graph = tmp.fill(BLDG) # tell 'fill' to put the invented names in the BLDG namespace

The `vav_graph` object can now be added to our model:

In [8]:
bldg.add_graph(vav_graph)

In [9]:
# run for the other n-1 VAVs
for vav in range(1,NUM_VAVs):
    vav_name = BLDG[f"vav-{vav}"]
    _, vav_graph = vav_templ.evaluate({"name": vav_name}).fill(BLDG)
    bldg.add_graph(vav_graph)

Print out the resulting model

In [10]:
print(bldg.graph.serialize(format="turtle"))

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<my-building> a owl:Ontology .

<urn:my-building/vav-0> a brick:VAV ;
    brick:feeds <urn:my-building/zone_8be47813> ;
    brick:hasPart <urn:my-building/dmp_9b8b98e4> ;
    brick:hasPoint <urn:my-building/co2_ccbb8751>,
        <urn:my-building/dat_cad14af4>,
        <urn:my-building/occ_dbe0fdb6>,
        <urn:my-building/ztemp_090e3792> .

<urn:my-building/vav-1> a brick:VAV ;
    brick:feeds <urn:my-building/zone_94658505> ;
    brick:hasPart <urn:my-building/dmp_aa5f3617> ;
    brick:hasPoint <urn:my-building/co2_f088b40b>,
        <urn:my-building/dat_8959bf92>,
        <urn:my-building/occ_d804c60f>,
        <urn:my-building/ztemp_72d2ddf7> .

<urn:my-building/vav-2> a brick:VAV ;
    brick:feeds <urn:my-building/zone_fd9654b3> ;
    brick:hasPart <urn:my-building/dmp_1aa63d9c> ;
    brick:hasPoint <urn:my-building/co2_034b64ad>,
        <urn:my-building/dat_7befac06>,
    

In [11]:
# and save your work!
bm.session.commit()

Rewinding a little bit, you may have noticed that the generated graph above doesn't actually contain any metadata for the points and parts associated with the VAVs. This is because we didn't deal with dependencies at all. We can use `inline_dependencies()` to resolve other definitions required by the VAV template:

In [12]:
# without resolving dependencies
_, g = vav_templ.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_cd57b406> a brick:VAV ;
    brick:feeds <urn:my-building/zone_4de18641> ;
    brick:hasPart <urn:my-building/dmp_7b0fd44b> ;
    brick:hasPoint <urn:my-building/co2_43920f5d>,
        <urn:my-building/dat_41a730c6>,
        <urn:my-building/occ_14aa04ca>,
        <urn:my-building/ztemp_10afbea7> .




In [13]:
# with resolved dependencies
inlined = vav_templ.inline_dependencies()
_, g = inlined.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_534325be> a brick:VAV ;
    brick:feeds <urn:my-building/zone_71908014> ;
    brick:hasPart <urn:my-building/dmp_8f4532e0> ;
    brick:hasPoint <urn:my-building/co2_6438b299>,
        <urn:my-building/dat_80c681b5>,
        <urn:my-building/occ_4ed4521d>,
        <urn:my-building/ztemp_fceb6e54> .

<urn:my-building/co2_6438b299> a brick:CO2_Level_Sensor .

<urn:my-building/dat_80c681b5> a brick:Discharge_Air_Temperature_Sensor .

<urn:my-building/dmp_8f4532e0> a brick:Damper ;
    brick:hasPoint <urn:my-building/dmppos_05bb94e4> .

<urn:my-building/dmppos_05bb94e4> a brick:Damper_Position_Command .

<urn:my-building/occ_4ed4521d> a brick:Occupancy_Sensor .

<urn:my-building/zone_71908014> a brick:HVAC_Zone .

<urn:my-building/ztemp_fceb6e54> a brick:Zone_Air_Temperature_Sensor .




In [14]:
print(inlined.body.serialize(format='turtle'))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#name> a brick:VAV ;
    brick:feeds <urn:___param___#zone> ;
    brick:hasPart <urn:___param___#dmp> ;
    brick:hasPoint <urn:___param___#co2>,
        <urn:___param___#dat>,
        <urn:___param___#occ>,
        <urn:___param___#ztemp> .

<urn:___param___#co2> a brick:CO2_Level_Sensor .

<urn:___param___#dat> a brick:Discharge_Air_Temperature_Sensor .

<urn:___param___#dmp> a brick:Damper ;
    brick:hasPoint <urn:___param___#dmppos> .

<urn:___param___#dmppos> a brick:Damper_Position_Command .

<urn:___param___#occ> a brick:Occupancy_Sensor .

<urn:___param___#zone> a brick:HVAC_Zone .

<urn:___param___#ztemp> a brick:Zone_Air_Temperature_Sensor .




Run another example w/ chiller plant

In [15]:
lib = Library.load(directory="../libraries/chiller-plant/")

In [16]:
chiller = lib.get_template_by_name("chiller")
inlined = chiller.inline_dependencies()
print(inlined.body.serialize())
_, g = inlined.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<urn:___param___#name> a brick:Chiller ;
    brick:hasPart <urn:___param___#chwp>,
        <urn:___param___#cnd>,
        <urn:___param___#cvlv> ;
    brick:hasPoint <urn:___param___#chwrt>,
        <urn:___param___#chwst> ;
    brick:isMeteredBy <urn:___param___#meter> .

<urn:___param___#chwp> a brick:Chilled_Water_Pump,
        brick:Pump ;
    brick:hasPart <urn:___param___#pump-bypass_valve>,
        <urn:___param___#pump-control_valve> ;
    brick:hasPoint <urn:___param___#pump-mode>,
        <urn:___param___#pump-run>,
        <urn:___param___#pump-vsd> ;
    brick:isMeteredBy <urn:___param___#pump-meter> .

<urn:___param___#chwrt> a brick:Chilled_Water_Return_Temperature_Sensor .

<urn:___param___#chwst> a brick:Chilled_Water_Supply_Temperature_Sensor .

<urn:___param___#cnd> a brick:Condenser ;
    brick:hasPart <urn:___param___#pump> ;
    brick:hasPoint <urn:___p